<a href="https://colab.research.google.com/github/aroslavakudrevic577-coder/python-ai-kudrevic-yaroslava/blob/main/notebooks/week2b_read_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

📊 Week 2: Data Analysis — Чтение и проверка данных о фильмах
Цель: Научиться читать CSV-файлы из репозитория GitHub в Google Colab и выполнять базовую проверку данных с помощью pandas.

Данные:
film_location_money.csv — информация о фильмах, локациях съёмок и кассовых сборах (box office)

Структура файла:
- film: URI фильма в Wikidata
- filmLabel: название фильма на русском языке
- locationLabel: локация съёмок
- boxOffice: кассовые сборы в долларах США

Что мы делаем:
- Клонируем ваш персональный репозиторий GitHub в Colab
- Читаем CSV-файл в pandas DataFrame
- Очищаем и анализируем структуру данных
- Проверяем корректность загрузки и типы данных

## 🐱 [1] Клонируем репозиторий курса в Colab

In [1]:
# 🎬 Шаг 1. Клонируем ваш персональный репозиторий в Colab

import os

if not os.path.exists("python-ai-kudrevic-yaroslava"):
    !git clone -q https://github.com/aroslavakudrevic577-coder/python-ai-kudrevic-yaroslava.git

%cd python-ai-kudrevic-yaroslava

print("✅ Репозиторий готов, теперь мы работаем внутри папки python-ai-kudrevic-yaroslava")

/content/python-ai-kudrevic-yaroslava
✅ Репозиторий готов, теперь мы работаем внутри папки python-ai-kudrevic-yaroslava


## 📥 [2A] Простое чтение CSV-файлов в pandas

Сначала просто прочитаем оба CSV-файла в объекты `DataFrame`, без каких‑либо изменений.

После этого мы узнаем, сколько строк загружено в каждый датасет.

In [2]:
# 🎬 Шаг 2. Чтение данных о фильмах в pandas

import pandas as pd

df_films = pd.read_csv("data/film_location_money.csv")

print("✅ Загружено строк в датасет:", len(df_films))
print("\n📋 Структура данных:")
print(df_films.info())

print("\n🔍 Первые 5 строк:")
print(df_films.head())

✅ Загружено строк в датасет: 8551

📋 Структура данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8551 entries, 0 to 8550
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   film           8551 non-null   object
 1   filmLabel      8551 non-null   object
 2   locationLabel  8551 non-null   object
 3   boxOffice      8551 non-null   object
dtypes: object(4)
memory usage: 267.3+ KB
None

🔍 Первые 5 строк:
                                      film                    filmLabel  \
0   http://www.wikidata.org/entity/Q685817                Преследование   
1  http://www.wikidata.org/entity/Q1168478   Безбрежное Саргассово море   
2  http://www.wikidata.org/entity/Q2893302                 Цена красоты   
3  http://www.wikidata.org/entity/Q3367212  Pas très normales activités   
4  http://www.wikidata.org/entity/Q4959480           Breakfast in Paris   

     locationLabel boxOffice  
0           Лондон     48500  
1 

## 🧹 [2B] Очистка и переименование столбцов

В исходном CSV-файле есть **технические столбцы**, которые полезны для Викиданных, но мешают простому анализу:

- Столбец `film` содержит URL-ссылку на объект в Wikidata — для анализа достаточно названия фильма.
- Столбцы `filmLabel`, `locationLabel` содержат человекочитаемые названия (фильм, локация съёмок).
- Столбец `boxOffice` хранит кассовые сборы в строковом формате.

В этом шаге мы:
- удалим технический столбец `film` (URL Wikidata);
- переименуем столбцы для удобства:  
  `filmLabel → film`,  
  `locationLabel → location`,  
  `boxOffice → box_office` (единый стиль: snake_case);
- приведём числовой столбец `box_office` к типу `int`.

При преобразовании чисел используем:
- `pd.to_numeric(..., errors="coerce")` — некорректные значения → `NaN`;
- `fillna(0)` — замена пропусков нулями;
- `astype(int)` — финальное приведение к целому числу.

> ⚠️ **Важно:** Удаление URL и нормализация имён столбцов критичны для дальнейшего анализа. Преобразование `box_office` в число позволяет строить графики и считать статистику.

In [3]:
# 🧹 Шаг 2B. Очистка и переименование столбцов (идемпотентная версия)

# 🔒 Проверка: существует ли переменная df_films
if 'df_films' not in globals() and 'df_films' not in locals():
    raise NameError(
        "❌ Ошибка: переменная 'df_films' не определена.\n"
        "Выполните сначала ячейку загрузки данных (Шаг 2)."
    )
if not isinstance(df_films, pd.DataFrame):
    raise TypeError("❌ Переменная 'df_films' существует, но не является pandas DataFrame.")

print("🔧 Начинаем очистку данных...")

# 🗑️ Удаляем столбец 'film' (URL Wikidata), только если он есть
if 'film' in df_films.columns:
    df_films = df_films.drop(columns=['film'])
    print("✅ Удалён технический столбец 'film' (URL Wikidata)")
else:
    print("ℹ️ Столбец 'film' отсутствует (уже удалён ранее или не существует)")

# 🏷️ Переименовываем столбцы: только если исходные имена существуют
rename_map = {}
if 'filmLabel' in df_films.columns:
    rename_map['filmLabel'] = 'film'
if 'locationLabel' in df_films.columns:
    rename_map['locationLabel'] = 'location'
if 'boxOffice' in df_films.columns:
    rename_map['boxOffice'] = 'box_office'

if rename_map:
    df_films = df_films.rename(columns=rename_map)
    print(f"✅ Переименованы столбцы: {rename_map}")
else:
    print("ℹ️ Столбцы для переименования отсутствуют (уже обработаны ранее)")

# 💰 Преобразуем кассовые сборы в целое число
target_col = None
if 'box_office' in df_films.columns:
    target_col = 'box_office'
elif 'boxOffice' in df_films.columns:  # Резервный вариант на случай сбоя переименования
    df_films = df_films.rename(columns={'boxOffice': 'box_office'})
    target_col = 'box_office'
    print("⚠️ Обнаружен необработанный столбец 'boxOffice' — переименован в 'box_office'")

if target_col:
    df_films['box_office'] = (
        pd.to_numeric(df_films['box_office'], errors='coerce')
        .fillna(0)
        .astype(int)
    )
    print("✅ Столбец 'box_office' приведён к типу int")
else:
    print("⚠️ Столбец с кассовыми сборами не найден. Преобразование пропущено.")

# 📋 Финальная проверка
print("\n" + "="*50)
print("✅ Очистка завершена успешно!")
print("\n📋 Итоговая структура данных:")
print(df_films.dtypes)
print(f"\n👀 Пример данных:\n{df_films.head(3)}")

🔧 Начинаем очистку данных...
✅ Удалён технический столбец 'film' (URL Wikidata)
✅ Переименованы столбцы: {'filmLabel': 'film', 'locationLabel': 'location', 'boxOffice': 'box_office'}
✅ Столбец 'box_office' приведён к типу int

✅ Очистка завершена успешно!

📋 Итоговая структура данных:
film          object
location      object
box_office     int64
dtype: object

👀 Пример данных:
                         film   location  box_office
0               Преследование     Лондон       48500
1  Безбрежное Саргассово море     Ямайка       45306
2                Цена красоты  Австралия       56101


## 🔍 [3] Обзор данных: структура и первые строки

Сделаем детальный обзор нашего единого DataFrame `df_films`:

- проверим размер таблицы (`shape`);
- выведем актуальные названия столбцов после очистки;
- посмотрим первые строки для визуальной проверки;
- проанализируем статистику по кассовым сборам (`box_office`): минимум, максимум, среднее, медиана.

Для удобства и читаемости используем вспомогательную функцию `show_info()`, которая структурирует вывод информации о любом DataFrame.

In [5]:
def show_info(df, name, n=5):
    """Краткий обзор DataFrame: имя, размер, столбцы, типы данных и первые строки."""
    print(f"\n{'='*50}")
    print(f"📊 {name}")
    print(f"{'='*50}")
    print(f"🔹 Размер таблицы: {df.shape[0]} строк × {df.shape[1]} столбцов")
    print(f"🔹 Столбцы: {', '.join(df.columns)}")
    print(f"\n🔹 Типы данных:")
    print(df.dtypes.to_string())
    print(f"\n🔹 Первые {n} строк:")
    print(df.head(n).to_string(index=False))

# 🔍 Шаг 3. Обзор данных (с защитой от ошибок)

# Проверка существования датафрейма
if 'df_films' not in globals() or not isinstance(df_films, pd.DataFrame):
    raise RuntimeError(
        "❌ Датафрейм 'df_films' не найден или повреждён.\n"
        "Выполните сначала ячейки: Шаг 2 (загрузка) → Шаг 2B (очистка)."
    )

# Проверка критически важного столбца
if 'box_office' not in df_films.columns:
    raise KeyError(
        "❌ Столбец 'box_office' отсутствует в данных.\n"
        "Убедитесь, что выполнена ячейка 'Шаг 2B. Очистка и переименование столбцов'."
    )

# Вывод информации
show_info(df_films, "Данные о фильмах, локациях съёмок и кассовых сборах", n=3)

# Статистика по кассовым сборам
print(f"\n{'='*50}")
print("📈 Статистика по кассовым сборам (box_office, USD):")
print(f"{'='*50}")
stats = df_films["box_office"].describe()
stats_formatted = stats.apply(
    lambda x: f"${x:,.0f}" if x >= 1000 else f"${x:.2f}" if x > 0 else "$0"
)
print(stats_formatted.to_string())

# Дополнительная полезная информация
unique_locations = df_films["location"].nunique()
top_location = df_films["location"].value_counts().index[0]
print(f"\n📍 Уникальных локаций съёмок: {unique_locations}")
print(f"📍 Самая частая локация: {top_location}")
print(f"\n✅ Обзор завершён. Данные готовы к анализу!")


📊 Данные о фильмах, локациях съёмок и кассовых сборах
🔹 Размер таблицы: 8551 строк × 3 столбцов
🔹 Столбцы: film, location, box_office

🔹 Типы данных:
film          object
location      object
box_office     int64

🔹 Первые 3 строк:
                      film  location  box_office
             Преследование    Лондон       48500
Безбрежное Саргассово море    Ямайка       45306
              Цена красоты Австралия       56101

📈 Статистика по кассовым сборам (box_office, USD):
count            $8,551
mean       $200,029,613
std        $283,278,148
min                  $0
25%         $23,650,000
50%         $95,708,457
75%        $262,552,893
max      $3,200,000,000

📍 Уникальных локаций съёмок: 1749
📍 Самая частая локация: Лос-Анджелес

✅ Обзор завершён. Данные готовы к анализу!


## ✅ [4] Быстрая проверка и валидация данных

Проведём базовую валидацию нашего датасета:

- сколько **уникальных фильмов** и **локаций съёмок** содержится в данных;
- **какие локации встречаются чаще всего** (Топ‑5 по числу фильмов);
- **статистика по кассовым сборам**: минимальные, максимальные и средние значения;
- **фильмы с самыми высокими кассовыми сборами** — для проверки корректности данных.

Функция `value_counts()`:
- подсчитывает частоту каждого уникального значения в столбце;
- автоматически сортирует результаты по убыванию.

Метод `.head(N)` возвращает первые N строк, поэтому  
`df_films["location"].value_counts().head(5)` покажет **Топ‑5 локаций по числу съёмок**.

In [6]:
# ✅ Шаг 4. Быстрая проверка и валидация данных

# 🔒 Проверка целостности данных
if 'df_films' not in globals() or not isinstance(df_films, pd.DataFrame):
    raise RuntimeError(
        "❌ Датафрейм 'df_films' не найден.\n"
        "Выполните сначала ячейки: Шаг 2 → Шаг 2B → Шаг 3."
    )

required_cols = ['film', 'location', 'box_office']
missing = [col for col in required_cols if col not in df_films.columns]
if missing:
    raise KeyError(
        f"❌ Отсутствуют обязательные столбцы: {', '.join(missing)}.\n"
        "Выполните ячейку 'Шаг 2B. Очистка и переименование столбцов'."
    )

print("🔍 Быстрая проверка данных о фильмах и локациях съёмок")
print("="*60)

# 🎬 Уникальные фильмы и локации
n_films = df_films["film"].nunique()
n_locations = df_films["location"].nunique()
total_rows = len(df_films)

print(f"\n🎬 Уникальных фильмов: {n_films}")
print(f"📍 Уникальных локаций съёмок: {n_locations}")
print(f"📊 Общее число записей: {total_rows}")

# 💰 Статистика по кассовым сборам
print(f"\n💰 Кассовые сборы (box_office, USD):")
box_min = df_films["box_office"].min()
box_max = df_films["box_office"].max()
box_mean = df_films["box_office"].mean()
box_median = df_films["box_office"].median()

print(f"   Минимум:   ${box_min:,.0f}")
print(f"   Максимум:  ${box_max:,.0f}")
print(f"   Среднее:   ${box_mean:,.0f}")
print(f"   Медиана:   ${box_median:,.0f}")

# 🌍 Топ-5 локаций по числу съёмок
print(f"\n🌍 Топ-5 локаций по числу фильмов:")
top_locations = df_films["location"].value_counts().head(5)
for i, (loc, count) in enumerate(top_locations.items(), 1):
    pct = count / total_rows * 100
    print(f"   {i}. {loc:<20} — {count} фильмов ({pct:.1f}%)")

# 🏆 Топ-5 фильмов по кассовым сборам
print(f"\n🏆 Топ-5 фильмов по кассовым сборам:")
top_films = df_films.nlargest(5, "box_office")[["film", "location", "box_office"]]
for i, (_, row) in enumerate(top_films.iterrows(), 1):
    print(f"   {i}. {row['film']:<30} ({row['location']}) — ${row['box_office']:>10,.0f}")

# ⚠️ Проверка на аномалии
if box_min < 0:
    print(f"\n⚠️  ВНИМАНИЕ: обнаружены отрицательные кассовые сборы (${box_min:,.0f}) — требуется проверка данных!")
if box_max > 1_000_000_000:
    print(f"⚠️  ВНИМАНИЕ: максимальные сборы превышают $1 млрд (${box_max:,.0f}) — проверьте корректность!")

print("\n" + "="*60)
print("✅ Валидация завершена. Данные готовы к анализу!")

🔍 Быстрая проверка данных о фильмах и локациях съёмок

🎬 Уникальных фильмов: 2438
📍 Уникальных локаций съёмок: 1749
📊 Общее число записей: 8551

💰 Кассовые сборы (box_office, USD):
   Минимум:   $0
   Максимум:  $3,200,000,000
   Среднее:   $200,029,613
   Медиана:   $95,708,457

🌍 Топ-5 локаций по числу фильмов:
   1. Лос-Анджелес         — 437 фильмов (5.1%)
   2. Нью-Йорк             — 381 фильмов (4.5%)
   3. Калифорния           — 326 фильмов (3.8%)
   4. Pinewood Studios     — 186 фильмов (2.2%)
   5. Лондон               — 174 фильмов (2.0%)

🏆 Топ-5 фильмов по кассовым сборам:
   1. Жил-был тигр                   (Дели) — $3,200,000,000
   2. Аватар                         (Калифорния) — $2,847,379,794
   3. Аватар                         (Новая Зеландия) — $2,847,379,794
   4. Аватар                         (Гавайи) — $2,847,379,794
   5. Мстители: Финал                (Глазго) — $2,797,501,328
⚠️  ВНИМАНИЕ: максимальные сборы превышают $1 млрд ($3,200,000,000) — проверьте кор

## 📝 Summary

**Что мы сделали в этом ноутбуке (Week 2):**

- ✅ Клонировали репозиторий GitHub в Colab
- ✅ Прочитали 2 CSV-файла из `data/examples/`
- ✅ Удалили URL Wikidata и переименовали столбцы (`*Label → короткие имена`)
- ✅ Проверили структуру данных (размер, столбцы, первые строки)
- ✅ Посмотрели базовую статистику по бюджету (`capital_cost`)
- ✅ Выполнили быструю валидацию:
  - количество уникальных фильмов, стран, жанров
  - диапазоны значений
  - топ стран и жанров по числу записей

Теперь у нас есть **аккуратные, проверенные таблицы**, с которыми удобно работать дальше.

В отдельном ноутбуке для следующей недели мы будем использовать **те же данные** для:
- более сложного анализа (группировки, фильтрация),
- и построения визуализаций (графики и диаграммы). 🎨